In [2]:
import requests
from collections import defaultdict, Counter


BASE_URL = 'https://api.github.com'
TOKEN = open('token.txt', 'r').read()


'''
# uncomment it if Counter would be bad
class MyDict(dict):
    def __add__(self, other):
        for key, value in other.items():
            if key in self.keys():
                self[key] += value
            else:
                self[key] = value
        return self
'''


def choose_user(all_users):
    print('Список всех доступных пользователей:\n',
          ', '.join(all_users), sep='', end='\n\n')
    username = input('Введите имя пользователя из этого списка: ')
    while username not in all_users:
        username = input('Такого пользователя нет в списке. Введите заново: ')
    return username


def get_repos(username):
    repos = []
    page = 1
    while True:
        curr_repos = requests.get(
            '{}/users/{}/repos?page={}&per_page=100&access_token={}'.format(BASE_URL, username, page, TOKEN)
        ).json()
        if not curr_repos:
            break

        repos += curr_repos
        page += 1
    return repos


def print_names_and_descr(repos):
    # красивый вывод названий и описаний репозиториев
    print('\nСписок репозиториев с описаниями:\n')
    for rep in repos:
        description = rep['description']
        if description is None:
            description = 'No description :('
        print(rep['name'], description, sep='\n', end='\n\n')


def count_languages(repos):
    languages_using = defaultdict(int)
    for rep in repos:
        languages_using[rep['language']] += 1
    try:
        del languages_using[None]  # отсутствие языка – не язык
    except KeyError:
        pass
    return languages_using


def get_followers(username):
    followers = []
    page = 1
    while True:
        curr_followers = requests.get(
            '{}/users/{}/followers?page={}&per_page=100&access_token={}'.format(BASE_URL, username, page, TOKEN)
        ).json()
        if not curr_followers:
            break
        followers += curr_followers
        page += 1
    return followers


def right_grammar_case(number):
    if number % 10 == 1:
        return 'репозитории'
    else:
        return 'репозиториях'

In [3]:
# в users.txt лежит список всех юзеров
with open('users.txt', 'r') as f:
    users = f.read().split()

# выбор пользователя из списка
username = choose_user(users)
print('Вы выбрали пользователя {}. Загружаю репозитории.'.format(username))

Список всех доступных пользователей:
elmiram, maryszmary, lizaku, nevmenandr, ancatmara, roctbb, akutuzov, agricolamz, lehkost, kylepjohnson, mikekestemont, demidovakatya, shwars, JelteF, timgraham, arogozhnikov, jasny, bcongdon, whyisjake

Введите имя пользователя из этого списка: elmiram
Вы выбрали пользователя elmiram. Загружаю репозитории.


In [4]:
# вывод названий и описаний
repos = get_repos(username)
print_names_and_descr(repos)


Список репозиториев с описаниями:

19thcentury
No description :(

2016learnpython
Python Teaching, Seminars for 2nd year students of School of Linguistics NRU HSE

2017learnpython
Python Teaching, Seminars for 3rd year students of School of Linguistics NRU HSE

animal_sounds_graph_project
No description :(

ASD_children_corpus
No description :(

children_texts_corpus
StartWrit corpus is a collection of texts written by elementary school children.

computer-science-flash-cards
Mini website for testing both general CS knowledge and enforce coding practice and common algorithm/data structure memorization.

constructicon
No description :(

elmiram.github.io
Resume

EmotionAnalysisInVideo
Tracker of basic emotions of a movie character built with Microsoft Computer Vision Services

google-interview-university
A complete daily plan for studying to become a Google software engineer.

HarryPotterBot
Bot uses words from Harry Potter books to generate phases.

hh-1-1-git-commands
No description 

In [5]:
languages_stat = count_languages(repos)

# все языки пользователя
print('{} пишет на'.format(username),
      ', '.join(languages_stat.keys()), end='.\n')

# статистика по использованию языков в репозиториях
for lang, counter in languages_stat.items():
    print('{} используется в {} {}.'.format(lang, counter, right_grammar_case(counter)))

elmiram пишет на JavaScript, Jupyter Notebook, CSS, HTML, Python, C#, Shell.
JavaScript используется в 6 репозиториях.
Jupyter Notebook используется в 2 репозиториях.
CSS используется в 1 репозитории.
HTML используется в 5 репозиториях.
Python используется в 11 репозитории.
C# используется в 1 репозитории.
Shell используется в 1 репозитории.


In [14]:
users_repos = {}
for user in users:
    users_repos[user] = get_repos(user)

In [15]:
# поиск пользователя с максимальным кол-вом репозиториев
max_user = max(users, key=(lambda user: len(users_repos[user])))
max_repos = len(users_repos[max_user])

print()
print('Больше всего ({}) репозиториев у {}.'.format(max_repos, max_user))


Больше всего (150) репозиториев у JelteF.


In [16]:
# поиск самого популярного языка
all_users_languages = sum((Counter(count_languages(users_repos[user])) for user in users), Counter())

'''
all_users_languages = Counter()
for user in users:
    curr_languages = count_languages(users_repos[user])
    all_users_languages += Counter(curr_languages)
'''

max_lang = max(all_users_languages, key=all_users_languages.get)
max_uses = all_users_languages[max_lang]
print('Самый популярный язык – {}.'.format(max_lang))

Самый популярный язык – Python.


In [17]:
# поиск наибольшего кол-ва подписчиков
max_followers = 0
max_user = ''
for user in users:
    curr_followers = get_followers(user)
    if len(curr_followers) > max_followers:
        max_followers = len(curr_followers)
        max_user = user
print('Больше всего подписчиков ({}) у {}'.format(max_followers, max_user))


Больше всего подписчиков (793) у timgraham
